# Baslangıc

In [1]:
!pip install tensorflow tensorflow-gpu pandas matplotlib 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.6 MB/s eta 0:00:00


In [ ]:
!pip list

In [2]:
import os 
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv(os.path.join('/content/drive/MyDrive/Yapay zeka toxic şu an/toxic (colab burda)/train.csv')) #model için kullanıcağım dosya nerede

In [5]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Yapay zeka toxic şu an/toxic (colab burda)/toxicity.h5')

# Hazırlama (Her özlelliği kullanmadan önce bu kısmı çalıştırın)

In [6]:
from tensorflow.keras.layers import TextVectorization

In [7]:
X = df['comment_text']
Y =df[df.columns[2:]].values

In [8]:
MAX_FEATURES = 200000 #modelin sözlüğündeki kelime sayısı

In [9]:
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')

In [10]:
vectorizer.adapt(X.values)

In [11]:
vectorized_text=vectorizer(X.values)

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [13]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

Derin Öğrenme Model Başlangıcı (Kendimiz oluşturup eğitmek için)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [ ]:
model2 = Sequential() 
model2.add(Embedding(MAX_FEATURES+1, 32))
model2.add(Bidirectional(LSTM(32, activation='tanh')))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(6, activation='sigmoid'))

In [ ]:
model2.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [ ]:
model2.summary()



In [ ]:
history = model2.fit(train, epochs=30, validation_data=val)

In [ ]:
from matplotlib import pyplot as plt
#Colabda çalışmaya bilir

In [ ]:
model2.save('toxicitykayıt.h5') #Modeli kaydetmek için

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

# Tahmin Test (Hazır model kullanıyorsanız esgeçebilirsiniz)



In [ ]:
input_text = vectorizer('Last chance stop or die')

In [ ]:
res = model.predict(np.array([input_text]))

In [ ]:
res

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)

In [ ]:
res.shape

# Doğrula Modeli Test et


In [ ]:
from tensorflow.keras.metrics import Precision, Recall,CategoricalAccuracy

In [ ]:
#Aşşağıdaki kodları 2. kere kullanmadan burdan bu değerleri sıfırlayın yoksa yanlış sonuç verir
pre=0
re=0
acc=0

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    # Pakedi aç
    X_true, y_true = batch
    # Tahmin yap eğer hazır model yerine kendiniz eğittiyseniz modelin ismini burdan ayarlayın
    ypred = model.predict(X_true)
    
    # Düzleştir
    y_true = y_true.flatten()
    ypred = ypred.flatten()
    
    pre.update_state(y_true, ypred)
    re.update_state(y_true, ypred)
    acc.update_state(y_true, ypred)
  

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.9927241206169128, Recall:0.9927241206169128, Accuracy:0.6509528756141663


# Gardio ,test ve tercüme

In [14]:
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 5.8 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [15]:
!pip install gradio jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [16]:
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
import gradio as gr
from deep_translator import GoogleTranslator
#Google traslateyi değiştirebilirsiniz

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 8s 8s/step


In [ ]:
res

array([[0.7979475 , 0.01220504, 0.3450206 , 0.01395374, 0.4024627 ,
        0.04953669]], dtype=float32)

In [20]:
def score_comment(comment):
    text=comment
    translated = GoogleTranslator(source='auto', target='en').translate(text=text)
    vectorized_comment = vectorizer([translated])
    results = model.predict(vectorized_comment)
    
    sayi=-100;
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
        if results[0][idx]>0.5:
          sayi+=1           
        if results[0][1]>0.5:  
            sayi+=3
    if sayi>=4:
      text+="Perma platformdan uzaklaştırma\n"
    if sayi==3:  
      text+="15 gün platformdan uzaklaştırma\n"
    if sayi==2:
      text+="3 gün platformdan uzaklaştırma uzaklaştırma\n"
    if sayi==1:
      text+="15 dakika platformdan uzaklaştırma uzaklaştırma\n"
    if sayi==0:
      text+="Bir ceza yok\n"

    return text

In [21]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=5, label="Buraya Yazınınz"),
                        outputs=gr.outputs.Textbox(label="Sonuc"))

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [22]:
interface.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Excel içindekileri oku yazdır


In [ ]:
import os 
import pandas as pd
from deep_translator import GoogleTranslator

In [ ]:
df2 = pd.read_csv(os.path.join('/content/drive/MyDrive/Yapay zeka toxic şu an/toxic (colab burda)/Comments.csv'))
#Comments kısmının altına istediğiniz kadar cümle ekleyebilirsiniz

In [ ]:
df2

,Comments
0,Bu gün harika bir gün.
1,Today is the day.
2,Fuck you! You pice of shit.
3,Senden nefret ediyorum.


In [ ]:
comments=df2['Comments']


In [ ]:
comments[1]

'Today is the day.'

In [ ]:
comments

0          Bu gün harika bir gün.
1               Today is the day.
2    Fuck you! You pice of  shit.
3         Senden nefret ediyorum.
Name: Comments, dtype: object

In [ ]:

Answers =[]

for i in range(comments.size):
    text = comments[i]
    translated = GoogleTranslator(source='auto', target='en').translate(text=text)
    vectorized_comment = vectorizer([translated])
    results = model.predict(vectorized_comment)
    text = '\n'
    sayi=0
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
        if results[0][idx]>0.5:
          sayi+=1           
        if results[0][1]>0.5:  
            sayi+=3
    if sayi>=4:
      text+="Perma platformdan uzaklaştırma\n"
    if sayi==3:  
      text+="15 gün platformdan uzaklaştırma\n"
    if sayi==2:
      text+="3 gün platformdan uzaklaştırma uzaklaştırma\n"
    if sayi==1:
      text+="15 dakika platformdan uzaklaştırma uzaklaştırma\n"
    if sayi==0:
      text+="Bir ceza yok\n"   
    Answers.append(text)

1/1 [==============================] - 0s 298ms/step


In [ ]:
df2['Answers']=Answers

In [ ]:
df2

,Comments,Answers
0,Bu gün harika bir gün.,\ntoxic: False\nsevere_toxic: False\nobscene: ...
1,Today is the day.,\ntoxic: False\nsevere_toxic: False\nobscene: ...
2,Fuck you! You pice of shit.,\ntoxic: True\nsevere_toxic: False\nobscene: T...
3,Senden nefret ediyorum.,\ntoxic: True\nsevere_toxic: False\nobscene: F...


In [ ]:
np.savetxt(r'np.txt', df2.values, fmt='%s', delimiter='\t')
#collabın içine kaydediyor drive değil